In [9]:
import os
import torch
import wandb
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import transforms
import sys
from datetime import datetime

### functions from _01_code/_99_common_utils/utils.py

In [11]:
def is_linux():
    if sys.platform.startswith("linux"):
        return True
    else:
        return False


def is_windows():
    if os.name == "nt":
        return True
    else:
        return False


def is_mac():
    if sys.platform == "darwin":
        return True
    else:
        return False


def get_num_cpu_cores():
    import multiprocessing
    return multiprocessing.cpu_count()

In [12]:
current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

config = {
    'epochs': 10_000,
    'batch_size': 2048,
    'learning_rate': 2e-5,
    'n_hidden_unit_list': [20, 20],
}

use_wandb = True

wandb.init(
    mode="online" if use_wandb else "disabled",
    project="CSE533_Fashion_MNIST",
    notes="Homework3",
    name=current_time_str,
    config=config
)

print(wandb.config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ckw6896 (dicelab). Use `wandb login --relogin` to force relogin


{'epochs': 10000, 'batch_size': 1024, 'learning_rate': 2e-05, 'n_hidden_unit_list': [20, 20]}


## [문제 1] Fashion MNIST 데이터 정규화를 위한 Mean과 Std값 찾기

In [16]:
data_path = "."

f_mnist_train = datasets.FashionMNIST(data_path, train=True, download=True, transform=transforms.ToTensor())
f_mnist_train, f_mnist_validation = random_split(f_mnist_train, [55_000, 5_000])

imgs = [img for img, _ in f_mnist_train]
imgs = torch.concat(imgs, dim=0)

mean = imgs.mean(dim=[0, 1, 2])
std = imgs.std(dim=[0, 1, 2])

num_data_loading_workers = get_num_cpu_cores() if is_linux() or is_windows() else 0

train_data_loader = DataLoader(
dataset=f_mnist_train, batch_size=wandb.config.batch_size, shuffle=True,
pin_memory=True, num_workers=num_data_loading_workers
)

validation_data_loader = DataLoader(
dataset=f_mnist_validation, batch_size=wandb.config.batch_size,
pin_memory=True, num_workers=num_data_loading_workers
)

f_mnist_transforms = nn.Sequential(
transforms.ConvertImageDtype(torch.float),
transforms.Normalize(mean=mean, std=std),
)

In [17]:
f_mnist_test_images = datasets.FashionMNIST(data_path, train=False, download=True)
f_mnist_test = datasets.FashionMNIST(data_path, train=False, download=True, transform=transforms.ToTensor())

test_data_loader = DataLoader(dataset=f_mnist_test, batch_size=len(f_mnist_test))

f_mnist_transforms = nn.Sequential(
transforms.ConvertImageDtype(torch.float),
transforms.Normalize(mean=mean, std=std),
)

## [문제 1] Answer

In [21]:
print("Fashion Mnist Train Data's Mean\t:", mean)
print("Fashion Mnist Train Data's Std\t:", std)

Fashion Mnist Train Data's Mean	:  tensor(0.2858)
Fashion Mnist Train Data's Std	:  tensor(0.3529)
